

![title](images/lostlake_gorgeflyshop.jpg)
Lost Lake, Mt Hood National Forest. Image credit: Gorge Fly Shop  

### Identify what it is we want to do  / loose requirements
* Want to know where we can go camping, lets start with near Mount Hood
* Start out with creating a dataframe we can query for specific characteristics
* What information will help us pick a campground?



### What are some camping analogies that might relate to what Im trying to convey about scalability?
* Send friends to multiple campgrounds to help find sites instead of everyone going to the same place - parallelization
* Come back to rendevous point because no cell service and compare what we found - 5 sites near water, no accesable sites, 
* Ikea camping chair?

In [1]:
from csv import DictReader
import geopandas as gpd
import json
import pandas as pd
import itertools

from camping.mocks.request import RequestsMock
from camping.util.scraper import Scraper
from camping.util.distance import distance_merge

def max_col_width(w=100):
    pd.set_option('display.max_colwidth', w)

ridb_facilities_url = "https://ridb.recreation.gov/api/v1/facilities"

#### RIDB - Recreation Information Database

[RIDB](https://ridb.recreation.gov/) is a data service spanning multiple US government agencies surfacing data related to recreational opportunities across the country.

##### Facilities endpoint: `https://ridb.recreation.gov/api/v1/facilities`
Get a list of facilities by activity. Camping is `activity_id=9`  
Query for facilities nearby a specific point via lat, long, and radius  
Query for facilities in a given state by supplying the 2 digit state code  

##### Campsites endpoint: `https://ridb.recreation.gov/api/v1/facilities{facility_id}/campsites`
Campsite specific information for a given facility. Returns a list of campsites at the facility

https://ridb.recreation.gov/docs  

Make sure you are making appropriate use of resources: 
https://ridb.recreation.gov/ridb-access-agreement 

RIDB has an API, which you can access with a Recreation.gov account. In this lab we will use a mock of the API, so you don't need an API key.

If you want to work with live data, go to the Profile area under your account and click the Generate New API key account. Place your key in the `apiKey` field of the `headers` dict below and replace `RequestsMock` with `requests` and you should be good to go!

Lets take a look at RIDB facilities with camping near Mount Hood Oregon

In [42]:
# NOTE: If using the mock not change params, mock looks for lat/long/radius
params = {"activity_id":9, "latitude":45.4977712, "longitude":-121.8211673, "radius":15}
headers = {"accept": "application/json", "apikey": "key"}
response = RequestsMock.get(ridb_facilities_url, params, headers=headers)
camping_json  = json.loads(response.text)
camping_json

{'RECDATA': [{'FacilityID': '234306',
   'LegacyFacilityID': '75167',
   'OrgFacilityID': 'AN375167',
   'ParentOrgID': '131',
   'ParentRecAreaID': '1102',
   'FacilityName': 'EAGLE CREEK OVERLOOK GRP SITE',
   'FacilityDescription': '<h2>Overview</h2>\nEagle Creek Overlook Group Site is set on a forested bluff above the Columbia River, providing an ideal setting for family gatherings and group events.<br/><br/>\n\nDeveloped by the Civilian Conservation Corps (CCC) in the 1930s as a place to view construction of the Bonneville Dam, this site features CCC masonry and offers expansive views of the Columbia River and mountains rising from the gorge.<h2>Recreation</h2>\n<p>The Eagle Recreation Area, just a short walk or bike ride away, provides visitors with opportunities for picnicking, hiking and wildlife viewing.<br><br> <br><br>Bonneville Dam, about a mile west of the campground, features sturgeon ponds, fish viewing and a visitor\'s center. Visitors will find display ponds showcasing

In [43]:
# Notice not all facilities are campgrounds
df_ridb_camping = pd.DataFrame(camping_json['RECDATA'])
df_ridb_camping.head(10)

,FacilityID,LegacyFacilityID,OrgFacilityID,ParentOrgID,ParentRecAreaID,FacilityName,FacilityDescription,FacilityTypeDescription,FacilityUseFeeDescription,FacilityDirections,...,FacilityMapURL,FacilityAdaAccess,GEOJSON,FacilityLongitude,FacilityLatitude,Keywords,StayLimit,Reservable,Enabled,LastUpdatedDate
0,234306,75167,AN375167,131,1102,EAGLE CREEK OVERLOOK GRP SITE,<h2>Overview</h2>\nEagle Creek Overlook Group ...,Campground,,"From Portland, OR; Travel east on Interstate 8...",...,,N,"{'TYPE': 'Point', 'COORDINATES': [-121.9308333...",-121.930833,45.641667,"ECOG,Overlook,Eagle Creek Overlook,Overlook Sh...",,True,True,2021-04-13
1,232834,71617,AN371617,131,1106,RILEY HORSE CAMPGROUND,<h2>Overview</h2>\nRiley Campground is an eque...,Campground,,"From Sandy, travel Highway 26 for 18 miles to ...",...,,N,"{'TYPE': 'Point', 'COORDINATES': [-121.8594444...",-121.859444,45.381389,"RILE,MT. HOOD NF - FS",,True,True,2021-04-13
2,234075,74082,AN374082,126,16835,WILDWOOD RECREATION SITE,<h2>Overview</h2>\n<p>Motorized access to <str...,Campground,<ul>\n<li>A daily vehicle fee is not included ...,Wildwood Recreation Site is located 39 miles e...,...,,N,"{'TYPE': 'Point', 'COORDINATES': [-121.9866667...",-121.986667,45.356111,"WIWO,Wildwood Recreation Site,Wild Wood Recrea...",,True,True,2021-04-13
3,233329,72357,AN372357,131,1106,TILLY JANE GUARD STATION,<h2>Overview</h2>\nTilly Jane Guard Station is...,Campground,<p>Reservation Fee: $6.00 (non-refundable)</p>...,"From Hood River, Oregon, travel south on Highw...",...,,N,"{'TYPE': 'Point', 'COORDINATES': [-121.6480556...",-121.648056,45.398889,"Guard Station,Cloud Cap Guard Station",,True,True,2021-04-13
4,122940,122940,,131,1102,WYETH CAMPGROUND,<h2>Overview</h2>\nWyeth Campground is just of...,Campground,,"From east or west on Oregon Interstate 84, tak...",...,,N,"{'TYPE': 'Point', 'COORDINATES': [-121.7722222...",-121.772222,45.690278,,,True,True,2021-04-13
5,10070260,,53640,131,,Vista Ridge Trailhead,<p> </p><p>No amenities. A popular trailhead t...,Facility,,"<p>From <u>Hood River, OR</u> - drive south on...",...,,,"{'TYPE': 'Point', 'COORDINATES': [-121.7068, 4...",-121.706800,45.462200,,,False,True,2020-09-10
6,272093,152792,AN452792,131,1106,SHERWOOD CAMPGROUND,<h2>Overview</h2>\nSherwood campground is an e...,Campground,,"From I-84 at Hood River, head south on highway...",...,,N,"{'TYPE': 'Point', 'COORDINATES': [-121.5703389...",-121.570339,45.394811,,,True,True,2021-04-13
7,251434,125541,AN425541,131,1106,LOST LAKE RESORT AND CAMPGROUND,<h2>Overview</h2>\n<p>Lost Lake Campground is ...,Campground,,Getting There:<br /> \nLost Lake Resort &amp; ...,...,,N,"{'TYPE': 'Point', 'COORDINATES': [-121.8219444...",-121.821944,45.488889,,,True,True,2021-04-13
8,232836,71619,AN371619,131,1106,TOLLGATE,<h2>Overview</h2>\nTollgate Campground is one ...,Campground,,"From Portland, travel southeast on Highway 26 ...",...,,N,"{'TYPE': 'Point', 'COORDINATES': [-121.9052778...",-121.905278,45.321944,"TOLL,MT. HOOD NF - FS",,True,True,2021-04-13
9,232835,71618,AN371618,131,1106,STILL CREEK,<h2>Overview</h2>\nStill Creek Campground lies...,Campground,,"From Portland, travel east on Highway 26 to Go...",...,,N,"{'TYPE': 'Point', 'COORDINATES': [-121.7355556...",-121.735556,45.295833,"STIC,MT. HOOD NF - FS",,True,True,2021-04-13


We have an idea of what we can get from the facilities endpoint:
* Facility Name & Facility ID
* Lat/Long
* Ada Accessability
* Description

Lets get more specific data on campsites at once of these facilities
Insert lost lake pic

In [44]:
df_ridb_camping.query("FacilityName == 'LOST LAKE RESORT AND CAMPGROUND'")['FacilityID']

7    251434
Name: FacilityID, dtype: object

In [45]:
resp = RequestsMock.get(f"{ridb_facilities_url}/251434/campsites", headers=headers)
resp.status_code

200

In [46]:
campsites = json.loads(resp.text)
df_campsites = pd.DataFrame(campsites['RECDATA'])
df_campsites.head()

,CampsiteID,FacilityID,CampsiteName,CampsiteType,TypeOfUse,Loop,CampsiteAccessible,CampsiteLongitude,CampsiteLatitude,CreatedDate,LastUpdatedDate,ATTRIBUTES,ENTITYMEDIA,PERMITTEDEQUIPMENT
0,96115,251434,A021,RV NONELECTRIC,Overnight,A Loop,False,-121.816561,45.498111,2015-04-09,2020-10-15,"[{'AttributeName': 'Driveway Grade', 'Attribut...",[{'EntityMediaID': '567f9beb-d16e-4317-b7e1-28...,"[{'EquipmentName': 'Trailer', 'MaxLength': 81}..."
1,96043,251434,C005,RV NONELECTRIC,Overnight,C Loop,True,-121.813871,45.496703,2015-04-09,2020-10-15,"[{'AttributeName': 'Driveway Entry', 'Attribut...",[{'EntityMediaID': '9d9d71ba-a3c2-41b8-b11c-b6...,"[{'EquipmentName': 'Trailer', 'MaxLength': 54}..."
2,96075,251434,D001,RV NONELECTRIC,Overnight,D Loop,True,-121.815180,45.494484,2015-04-09,2020-10-15,"[{'AttributeName': 'Accessibility', 'Attribute...",[{'EntityMediaID': '7c833b0a-b0cd-4093-8c91-95...,"[{'EquipmentName': 'Trailer', 'MaxLength': 36}..."
3,96104,251434,A009,RV NONELECTRIC,Overnight,A Loop,False,-121.816130,45.498334,2015-04-09,2020-10-15,"[{'AttributeName': 'Base Number of People', 'A...",[{'EntityMediaID': 'd4cec022-4663-4245-a823-a0...,"[{'EquipmentName': 'Trailer', 'MaxLength': 34}..."
4,96032,251434,H001,EQUESTRIAN NONELECTRIC,Overnight,Horsecamp,True,-121.811069,45.484083,2015-04-09,2020-10-15,"[{'AttributeName': 'Horse Stall/Corral', 'Attr...",[{'EntityMediaID': '2590bc58-4289-4a98-934a-6f...,"[{'EquipmentName': 'Horse', 'MaxLength': 32}]"


In [47]:
[entry for entry in df_campsites.iloc[0].ATTRIBUTES]

[{'AttributeName': 'Driveway Grade', 'AttributeValue': 'Slight'},
 {'AttributeName': 'Max Num of Vehicles', 'AttributeValue': '1'},
 {'AttributeName': 'Fire Pit', 'AttributeValue': 'Y'},
 {'AttributeName': 'Privacy', 'AttributeValue': 'Y'},
 {'AttributeName': 'Base Number of Vehicles', 'AttributeValue': '0'},
 {'AttributeName': 'Base Number of People', 'AttributeValue': '0'},
 {'AttributeName': 'Grills/Fire Ring', 'AttributeValue': 'Y'},
 {'AttributeName': 'Placed on Map', 'AttributeValue': '1'},
 {'AttributeName': 'Picnic Table', 'AttributeValue': 'Y'},
 {'AttributeName': 'Checkin Time', 'AttributeValue': '3:00 PM'},
 {'AttributeName': 'Location Rating', 'AttributeValue': 'Good'},
 {'AttributeName': 'Site Rating', 'AttributeValue': 'Preferred'},
 {'AttributeName': 'Capacity/Size Rating', 'AttributeValue': 'Single'},
 {'AttributeName': 'Checkout Time', 'AttributeValue': '10:00 AM'},
 {'AttributeName': 'Max Num of People', 'AttributeValue': '5'},
 {'AttributeName': 'Campfire Allowed', '

Do the campsite attributes have the information we are looking for?

In [48]:
ridb_attributes = set(itertools.chain(*df_campsites['ATTRIBUTES'].apply(lambda x: [entry['AttributeName'] for entry in x])))
ridb_attributes

{'Accessibility',
 'BBQ',
 'Base Number of People',
 'Base Number of Vehicles',
 'Campfire Allowed',
 'Capacity/Size Rating',
 'Checkin Time',
 'Checkout Time',
 'Double Driveway',
 'Driveway Entry',
 'Driveway Grade',
 'Driveway Length',
 'Driveway Surface',
 'Fire Pit',
 'Grills/Fire Ring',
 'Hike In Distance to Site',
 'Horse Hitching Post',
 'Horse Stall/Corral',
 'IS EQUIPMENT MANDATORY',
 'Location Rating',
 'Max Num of People',
 'Max Num of Vehicles',
 'Max Vehicle Length',
 'Min Num of People',
 'Pets Allowed',
 'Picnic Table',
 'Placed on Map',
 'Platform',
 'Privacy',
 'Proximity to Water',
 'Quiet Area',
 'Shade',
 'Site Access',
 'Site Height/Overhead Clearance',
 'Site Rating',
 'Tent Pad',
 'Tent Pad Length',
 'Tent Pad Width'}

In [49]:
# Note that boolean attributes are filled if they have a truthy value
attribute_name = 'proximity to water'
for campground in df_campsites['ATTRIBUTES']:
    for attribute in campground:
        if attribute['AttributeName'].lower() == attribute_name:
            print(attribute['AttributeValue'])

Lakefront
Lakefront
Lakefront
Lakefront
Lakefront


In [ ]:
json_normalize()

In [ ]:
# This is a lot of computation to check, any ideas on how we can improve this runtime?

In [35]:
df_campsites['AttributeDict'] = df_campsites['ATTRIBUTES'].apply(lambda x: {item['AttributeName']: item['AttributeValue'] for item in x})

In [41]:
# vectorized - WIP
def to_dict(attributes):
    result = {}
    for entry in 
    return {item['AttributeName']: item['AttributeValue'] for item in attributes[0]}

to_dict(df_campsites['ATTRIBUTES'].values)

{'Driveway Grade': 'Slight',
 'Max Num of Vehicles': '1',
 'Fire Pit': 'Y',
 'Privacy': 'Y',
 'Base Number of Vehicles': '0',
 'Base Number of People': '0',
 'Grills/Fire Ring': 'Y',
 'Placed on Map': '1',
 'Picnic Table': 'Y',
 'Checkin Time': '3:00 PM',
 'Location Rating': 'Good',
 'Site Rating': 'Preferred',
 'Capacity/Size Rating': 'Single',
 'Checkout Time': '10:00 AM',
 'Max Num of People': '5',
 'Campfire Allowed': 'Yes',
 'Pets Allowed': 'Yes',
 'Shade': 'Yes',
 'Tent Pad Length': '6',
 'Tent Pad Width': '6',
 'IS EQUIPMENT MANDATORY': 'true',
 'Driveway Surface': 'Paved',
 'Driveway Length': '93',
 'Driveway Entry': 'Pull-Through',
 'Max Vehicle Length': '81',
 'Site Height/Overhead Clearance': '50',
 'Tent Pad': 'Y',
 'Quiet Area': 'Y',
 'Hike In Distance to Site': '0',
 'Site Access': 'Drive-In'}

In [ ]:
# Combine the campground attributes with the facility data for 1 large denormalized table to query
max_col_width()
df_combined = df_campsites[['FacilityID', 'CampsiteID', 'CampsiteName', 'ATTRIBUTES']].merge(df_ridb_camping, on='FacilityID', how='left')
df_combined.head()

In [ ]:
def query(attributes, fields):
    found = 0
    for attribute in attributes:
        if attribute['AttributeName'] in fields: # and attribute['AttributeName'] is not None:
            found+=1
    if found == len(fields):
        return True
    return False

In [ ]:
# refer to ridb_attributes for field options
fields = ['Accessibility', 'Proximity to Water']
df_combined['Match'] = df_combined['ATTRIBUTES'].apply(lambda x : query(x, fields))
df_combined.query("Match == True")

In [2]:
# putting it all together
ridb_facilities_url = "https://ridb.recreation.gov/api/v1/facilities"
params = {"activity_id":9, "state":"OR"}
headers = {"accept": "application/json", "apikey": "key"}


response = RequestsMock.get(ridb_facilities_url, params, headers=headers)
camping_json  = json.loads(response.text)

# Do we really need the campgrounds in a dataframe?
df_ridb_camping = pd.DataFrame(camping_json['RECDATA'])

campground_info = pd.DataFrame()
for facility in camping_json['RECDATA']:
    if facility.get('FacilityID') is not None:
        campground_url = f"{ridb_facilities_url}/{facility['FacilityID']}/campsites"
        resp = RequestsMock.get(campground_url, headers=headers)
        if resp.status_code != 200:
            continue
        
        campsites = json.loads(resp.text)
        if len(campsites['RECDATA']) > 0:
            df_campsites = pd.DataFrame(campsites['RECDATA'])
            campground_info = campground_info.append(df_campsites[['FacilityID', 'CampsiteID', 'CampsiteName', 'ATTRIBUTES']].merge(df_ridb_camping, on='FacilityID', how='left'))

In [ ]:
campground_info['FacilityName'].unique()

At this point we have: 
* Ability to search for site characteristics
* Campground location and site name

Nice to have:
* Water availabilty
* Restroom access
* Current status - may be in facility description but not always

In [ ]:
sc = Scraper("http://www.fs.usda.gov/recarea/mthood/recreation/camping-cabins/recarea/?recid=53228&actid=29", "Lost Lake")
sc.scrape()

In [3]:
nf_sites = []
with open('../data/NF_sites/OR_sitelist.csv') as f:
    reader = DictReader(f)
    for row in reader:
        nf_sites.append(row)
nf_sites

[{'site_name': 'East Lemolo Campground',
  'site_url': 'https://www.fs.usda.gov/recarea/umpqua/recarea/?recid=63492'},
 {'site_name': 'Magone Lake Campground',
  'site_url': 'https://www.fs.usda.gov/recarea/malheur/recarea/?recid=39964'},
 {'site_name': 'East Davis Lake Campground',
  'site_url': 'https://www.fs.usda.gov/recarea/deschutes/recarea/?recid=38854'},
 {'site_name': 'Lost Lake Campground Resort and Day Use Area',
  'site_url': 'https://www.fs.usda.gov/recarea/mthood/recarea/?recid=53228'},
 {'site_name': 'Anthony Lake',
  'site_url': 'https://www.fs.usda.gov/recarea/wallowa-whitman/recarea/?recid=52199'},
 {'site_name': 'Musick Guard Station',
  'site_url': 'https://www.fs.usda.gov/recarea/umpqua/recarea/?recid=63428'},
 {'site_name': 'Lost Lake Campground',
  'site_url': 'https://www.fs.usda.gov/recarea/willamette/recarea/?recid=13362'}]

In [4]:
nf_data = []
for site in nf_sites:
    sc = Scraper(site['site_url'], site['site_name'])
    nf_data.append(sc.scrape())
nf_df = pd.DataFrame(nf_data)
nf_df

,FacilityStatus,FacilityLatitude,FacilityLongitude,FacilityElevation,Conditions,Reservations,FacilityName,Water,Restroom,Open Season
0,Temporarily Closed,43.310697,-122.162651,"4,150 feet",10/28/2020: Closed for the season. Will reopen...,Reservations can be made at www.recreation.gov...,East Lemolo Campground,NaN,NaN,NaN
1,Open,44.55266,-118.9094,5500,01/22/2021: The campground is is closed and th...,"To reserve the group site, visit www.recreatio...",Magone Lake Campground,Drinking Water,Vault Toilets,NaN
2,Closed,43.5867,-121.85667,4400,NaN,Reservations can be online through Recreation....,East Davis Lake Campground,Potable Water,Vault Toilet,NaN
3,Closed,45.50080,-121.81641,3200,CLOSED FOR THE SEASON\n \n**Lost Lake is curre...,Reservations can be made by visiting Recreatio...,Lost Lake Campground Resort and Day Use Area,Drinking Water,Vault Toilet (18),NaN
4,Closed,44.9625128531073,-118.228574730768,7150,Current Conditions,https://anthonylakes.com/campgrounds/,Anthony Lake,Potable Water,Vault Toilets,July - September
5,Temporarily Closed,43.581026,-122.641745,"5,000 feet",10/09/2020- This site is currently closed per ...,NaN,Musick Guard Station,NaN,NaN,Early Summer
6,Temporarily Closed,44.42927714677809,-121.912474623539,4200 feet,NaN,No advance reservations. All sites are first c...,Lost Lake Campground,NaN,NaN,- late-October (dependent on weather)


In [ ]:

campground_info.shape

In [13]:
dm = distance_merge(nf_df, campground_info, 1500, 'ridb', 'nf')

/Users/gizmo/.pyenv/versions/3.8.5/envs/strata_venv/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [14]:
dm.FacilityName_nf.unique()

array([nan, 'Magone Lake Campground', 'East Davis Lake Campground',
       'Anthony Lake', 'Musick Guard Station'], dtype=object)

In [22]:
# putting it all together
ridb_facilities_url = "https://ridb.recreation.gov/api/v1/facilities"
params = {"activity_id":9, "state":"OR"}
headers = {"accept": "application/json", "apikey": "key"}


response = RequestsMock.get(ridb_facilities_url, params, headers=headers)
camping_json  = json.loads(response.text)

# Do we really need the campgrounds in a dataframe?
df_ridb_camping = pd.DataFrame(camping_json['RECDATA'])

campground_info = pd.DataFrame()
for facility in camping_json['RECDATA']:
    if facility.get('FacilityID') is not None:
        campground_url = f"{ridb_facilities_url}/{facility['FacilityID']}/campsites"
        resp = RequestsMock.get(campground_url, headers=headers)
        if resp.status_code != 200:
            continue
        
        campsites = json.loads(resp.text)
        if len(campsites['RECDATA']) > 0:
            df_campsites = pd.DataFrame(campsites['RECDATA'])
            campground_info = campground_info.append(df_campsites[['FacilityID', 'CampsiteID', 'CampsiteName', 'ATTRIBUTES']].merge(df_ridb_camping, on='FacilityID', how='left'))
            
nf_data = []
with open('../data/NF_sites/OR_sitelist.csv') as f:
    reader = DictReader(f)
    for row in reader:
        sc = Scraper(row['site_url'], row['site_name'])
        nf_data.append(sc.scrape())
nf_df = pd.DataFrame(nf_data)
merged = distance_merge(nf_df, campground_info, 2000, 'ridb', 'nf')

/Users/gizmo/.pyenv/versions/3.8.5/envs/strata_venv/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/Users/gizmo/.pyenv/versions/3.8.5/envs/strata_venv/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [23]:
merged

,FacilityID,CampsiteID,CampsiteName,ATTRIBUTES,LegacyFacilityID,OrgFacilityID,ParentOrgID,ParentRecAreaID,FacilityName_ridb,FacilityDescription,...,FacilityStatus,FacilityLatitude_nf,FacilityLongitude_nf,FacilityElevation,Conditions,Reservations,FacilityName_nf,Water,Restroom,Open Season
0,251894,98358,008,"[{'AttributeName': 'Location Rating', 'Attribu...",135642,AN435642,131,1112,EAST LEMOLO CAMPGROUND,<h2>Overview</h2>\nEast Lemolo is on the banks...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,251894,98441,014,"[{'AttributeName': 'Picnic Table', 'AttributeV...",135642,AN435642,131,1112,EAST LEMOLO CAMPGROUND,<h2>Overview</h2>\nEast Lemolo is on the banks...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,251894,98438,004,"[{'AttributeName': 'Picnic Table', 'AttributeV...",135642,AN435642,131,1112,EAST LEMOLO CAMPGROUND,<h2>Overview</h2>\nEast Lemolo is on the banks...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,251894,98389,006,"[{'AttributeName': 'Picnic Table', 'AttributeV...",135642,AN435642,131,1112,EAST LEMOLO CAMPGROUND,<h2>Overview</h2>\nEast Lemolo is on the banks...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,251894,98359,005,"[{'AttributeName': 'Placed on Map', 'Attribute...",135642,AN435642,131,1112,EAST LEMOLO CAMPGROUND,<h2>Overview</h2>\nEast Lemolo is on the banks...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,251434,96303,F001,"[{'AttributeName': 'Checkout Time', 'Attribute...",125541,AN425541,131,1106,LOST LAKE RESORT AND CAMPGROUND,<h2>Overview</h2>\n<p>Lost Lake Campground is ...,...,Closed,45.50080,-121.81641,3200,CLOSED FOR THE SEASON\n \n**Lost Lake is curre...,Reservations can be made by visiting Recreatio...,Lost Lake Campground Resort and Day Use Area,Drinking Water,Vault Toilet (18),NaN
46,251434,96053,B011,"[{'AttributeName': 'Checkout Time', 'Attribute...",125541,AN425541,131,1106,LOST LAKE RESORT AND CAMPGROUND,<h2>Overview</h2>\n<p>Lost Lake Campground is ...,...,Closed,45.50080,-121.81641,3200,CLOSED FOR THE SEASON\n \n**Lost Lake is curre...,Reservations can be made by visiting Recreatio...,Lost Lake Campground Resort and Day Use Area,Drinking Water,Vault Toilet (18),NaN
47,251434,96013,B002,"[{'AttributeName': 'Driveway Length', 'Attribu...",125541,AN425541,131,1106,LOST LAKE RESORT AND CAMPGROUND,<h2>Overview</h2>\n<p>Lost Lake Campground is ...,...,Closed,45.50080,-121.81641,3200,CLOSED FOR THE SEASON\n \n**Lost Lake is curre...,Reservations can be made by visiting Recreatio...,Lost Lake Campground Resort and Day Use Area,Drinking Water,Vault Toilet (18),NaN
48,251434,96009,D004,"[{'AttributeName': 'Grills/Fire Ring', 'Attrib...",125541,AN425541,131,1106,LOST LAKE RESORT AND CAMPGROUND,<h2>Overview</h2>\n<p>Lost Lake Campground is ...,...,Closed,45.50080,-121.81641,3200,CLOSED FOR THE SEASON\n \n**Lost Lake is curre...,Reservations can be made by visiting Recreatio...,Lost Lake Campground Resort and Day Use Area,Drinking Water,Vault Toilet (18),NaN
